In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *

In [2]:
WINDOW_SIZE = 24

In [14]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
raw_data = DataLabeling(df, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.TI_data, rolling=10)
Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)

In [28]:
model = MLP_model(24, 43)

In [29]:
test_model = model.model

In [30]:
test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path='with_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/20000
11866/11866 [==============================] - 203s 17ms/step - loss: 0.7815 - accuracy: 0.5192 - val_loss: 0.8176 - val_accuracy: 0.0000e+00
Epoch 2/20000
11866/11866 [==============================] - 205s 17ms/step - loss: 0.7598 - accuracy: 0.5225 - val_loss: 0.7959 - val_accuracy: 0.0000e+00
Epoch 3/20000
11866/11866 [==============================] - 201s 17ms/step - loss: 0.7581 - accuracy: 0.5210 - val_loss: 0.7844 - val_accuracy: 0.0000e+00
Epoch 4/20000
11866/11866 [==============================] - 201s 17ms/step - loss: 0.7586 - accuracy: 0.5208 - val_loss: 0.7845 - val_accuracy: 0.0000e+00
Epoch 5/20000
11866/11866 [==============================] - 202s 17ms/step - loss: 0.7579 - accuracy: 0.5211 - val_loss: 0.7953 - val_accuracy: 0.0000e+00
Epoch 6/20000
11866/11866 [==============================] - 200s 17ms/step - loss: 0.7589 - accuracy: 0.5202 - val_loss: 0.7854 - val_accuracy: 0.0000e+00
Epoch 7/20000
11866/11866 [==============================] - 200

In [37]:
test_model = tf.keras.models.load_model('with_TIs\MLP_model')
test_model.evaluate(Data.test)

7/7 [==============================] - 0s 8ms/step - loss: 0.7314 - accuracy: 0.3562


[0.7313848733901978, 0.3561643958091736]

In [35]:
tf.argmax(test_model.predict(Data.val), axis=1)

<tf.Tensor: shape=(169,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [34]:
test_df.Trend.value_counts()

2    69
0    27
Name: Trend, dtype: int64